Now that we Crawled and collected data as csv files we
wanna create a database with "MySQL" and "MySQL connector"

# Import libraries

In [23]:
import streamlit as st
from pathlib import Path
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
import matplotlib.pyplot as plt 
import numpy as np
import altair as alt
import csv
import pandas as pd

first we should Create a user and grant all previliges to it in MySQL

Now that we Created user we should connect python to MySQL with MySQL connector

create a mysql connection using `mysql.connector.connect` function and pass initial configs!

In [5]:
cnx = mysql.connector.connect(
    user="fateme_mousavi",
    host="127.0.0.1", 
    password="FMousavi11121375",
)
cursor = cnx.cursor()

# DataBase Creation

In [6]:
def create_database(cursor, DB_NAME): 
    try:
        cursor.execute(f"DROP DATABASE IF EXISTS {DB_NAME}")
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME} DEFAULT CHARACTER SET 'utf8'")
        cursor.reset()
        print(f"Successfully created database: {DB_NAME}")
    except mysql.connector.Error as err:
        print(f"Failed creating database: {err}")


In [7]:
DB_NAME = 'IMDB'
create_database(cursor, DB_NAME)

Successfully created database: IMDB


Check database name in our localhost 

In [8]:
cursor.execute("SHOW DATABASES;")
DBs = tuple()
for x in cursor:
    DBs += x

In [9]:
for i, x in enumerate(DBs):
    print(f"{i}\t{x}")

0	contest
1	imdb
2	information_schema
3	mahex
4	mysql
5	performance_schema
6	sakila
7	superstore
8	sys
9	world


# Tables Creation

First table: movie

In [10]:
TABLES = {}
TABLES['movie'] = (
    "CREATE TABLE IF NOT EXISTS `movie` ("
    "  `id` varchar(8) NOT NULL,"
    "  `title` varchar(128),"
    "  `year` int(11) ,"
    "  `runtime` int(11) ,"
    "  `parental_guide` varchar(8) NOT NULL,"
    "  `gross_us_canada` int(11),"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

Second Table: Person

In [11]:
TABLES['person'] = (
    "CREATE TABLE IF NOT EXISTS `person` ("
    "  `id` varchar(8) NOT NULL,"
    "  `name` varchar(32),"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

Third table: Cast

In [13]:
TABLES['cast'] = (
    "CREATE TABLE IF NOT EXISTS `cast` ("
    "`id` INT NOT NULL AUTO_INCREMENT,"
    " `movie_id` varchar(8) NOT NULL,"
    " `person_id` varchar(8) NOT NULL,"
    "  PRIMARY KEY (`id`),"
    "  CONSTRAINT `cast_ibfk_1` FOREIGN KEY (`person_id`) "
    "     REFERENCES `person` (`id`) ON DELETE CASCADE,"
    "  CONSTRAINT `cast_ibfk_2` FOREIGN KEY (`movie_id`) "
    "     REFERENCES `movie` (`id`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

fourth table: Crew

In [14]:
TABLES['crew'] = (
    "CREATE TABLE IF NOT EXISTS `crew` ("
    "`id` INT NOT NULL AUTO_INCREMENT,"
    " `movie_id` varchar(8) NOT NULL,"
    " `person_id` varchar(8) NOT NULL,"
    " `role` varchar(8) NOT NULL,"
    "  PRIMARY KEY (`id`),"
    "  CONSTRAINT `crew_ibfk_1` FOREIGN KEY (`person_id`) "
    "     REFERENCES `person` (`id`) ON DELETE CASCADE,"
    "  CONSTRAINT `crew_ibfk_2` FOREIGN KEY (`movie_id`) "
    "     REFERENCES `movie` (`id`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

Last table: Genre

In [16]:
TABLES['genre'] = (
    "CREATE TABLE IF NOT EXISTS `genre` ("
    "`id` INT NOT NULL AUTO_INCREMENT,"
    " `movie_id` varchar(8) NOT NULL,"
    " `genre` varchar(16) NOT NULL,"
    "  PRIMARY KEY (`id`),"
    "  CONSTRAINT `genre_ibfk_2` FOREIGN KEY (`movie_id`) "
    "     REFERENCES `movie` (`id`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")


In [17]:
cursor.execute(f"USE {DB_NAME}")

In [18]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


Creating table movie: OK
Creating table person: OK
Creating table cast: OK
Creating table crew: OK
Creating table genre: OK


Show tables that we created

In [19]:
cursor = cnx.cursor()
cursor.execute(f"USE {DB_NAME};")
cursor.execute("SHOW TABLES;")
for x in cursor:
    print(x)

('cast',)
('crew',)
('genre',)
('movie',)
('person',)


# Data Insertion

Insert Movie table Data from CSV file

In [24]:
with open('movie.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ',')
    for row in reader:
        print(row['id'], row['title'], row['year'],row['runtime'],row['parental_guide'],row['gross_us_canada'])
        # insert

        sql_statement =('INSERT INTO movie'
                       '(id,title, year, runtime, parental_guide, gross_us_canada)'
                        'VALUES(%(id)s, %(title)s, %(year)s, %(runtime)s, %(parental_guide)s, %(gross_us_canada)s)')
        
        cursor.execute(sql_statement,row)
cnx.commit()

0111161 The Shawshank Redemption 1994 142 R 28767189
0068646 The Godfather 1972 175 R 136381073
0468569 The Dark Knight 2008 152 PG-13 534987076
0071562 The Godfather Part II 1974 202 R 47834595
0050083 12 Angry Men 1957 96 Approved 0
0108052 Schindler's List 1993 195 R 96898818
0167260 The Lord of the Rings: The Return of the King 2003 201 PG-13 379427292
0110912 Pulp Fiction 1994 154 R 107928762
0120737 The Lord of the Rings: The Fellowship of the Ring 2001 178 PG-13 316115420
0060196 The Good, the Bad and the Ugly 1966 178 Approved 25100000
0109830 Forrest Gump 1994 142 PG-13 330455270
0137523 Fight Club 1999 139 R 37030102
0167261 The Lord of the Rings: The Two Towers 2002 179 PG-13 342952511
1375666 Inception 2010 148 PG-13 292587330
0080684 Star Wars: Episode V - The Empire Strikes Back 1980 124 PG 292753960
0133093 The Matrix 1999 136 R 172076928
0099685 Goodfellas 1990 145 R 46909721
0073486 One Flew Over the Cuckoo's Nest 1975 133 R 108981275
0114369 Se7en 1995 127 R 100125643

Query to show Movie table

In [25]:
cursor.execute('select * from movie order by runtime;')

In [26]:
records = []
for item in cursor.fetchall() :
    print(item)
    records.append(item)

('0015324', 'Sherlock Jr.', 1924, 45, 'Passed', 977375)
('0017925', 'The General', 1926, 67, 'Passed', 0)
('0012349', 'The Kid', 1921, 68, 'Passed', 0)
('0381681', 'Before Sunset', 2004, 80, 'R', 5820649)
('0114709', 'Toy Story', 1995, 81, 'G', 223225679)
('0060827', 'Persona', 1966, 83, 'Unrated', 0)
('0096283', 'My Neighbor Totoro', 1988, 86, 'G', 2250213)
('0129167', 'The Iron Giant', 1999, 86, 'PG', 23315035)
('0021749', 'City Lights', 1931, 87, 'G', 19181)
('0027977', 'Modern Times', 1936, 87, 'G', 163577)
('0042876', 'Rashomon', 1950, 88, 'Unrated', 46808)
('0050825', 'Paths of Glory', 1957, 88, 'Approved', 0)
('0110357', 'The Lion King', 1994, 88, 'G', 422783777)
('0040522', 'Bicycle Thieves', 1948, 89, 'Unrated', 371111)
('0092005', 'Stand by Me', 1986, 89, 'R', 52287414)
('0095327', 'Grave of the Fireflies', 1988, 89, 'Unrated', 516962)
('0118849', 'Children of Heaven', 1997, 89, 'PG', 933933)
('0103639', 'Aladdin', 1992, 90, 'G', 217350219)
('0050986', 'Wild Strawberries', 19

In [27]:
len(records)

250

# Insert Person table data from CSV

In [28]:
with open('person.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ',')
    for row in reader:
        print(row['person_id'], row['name'])
        # insert

        sql_statement =('INSERT INTO `person`'
                       '(`id`, `name`)'
                        'VALUES(%(person_id)s, %(name)s)')
        
        cursor.execute(sql_statement,row)
cnx.commit()

0001104 Frank Darabont
0000175 Stephen King
0000209 Tim Robbins
0000151 Morgan Freeman
0348409 Bob Gunton
0000338 Francis Ford Coppola
0701374 Mario Puzo
0000008 Marlon Brando
0000199 Al Pacino
0001001 James Caan
0634240 Christopher Nolan
0634300 Jonathan Nolan
0275286 David S. Goyer
0000288 Christian Bale
0005132 Heath Ledger
0001173 Aaron Eckhart
0000134 Robert De Niro
0000380 Robert Duvall
0001486 Sidney Lumet
0741627 Reginald Rose
0000020 Henry Fonda
0002011 Lee J. Cobb
0000842 Martin Balsam
0000229 Steven Spielberg
0447745 Thomas Keneally
0001873 Steven Zaillian
0000553 Liam Neeson
0000146 Ralph Fiennes
0001426 Ben Kingsley
0001392 Peter Jackson
0866058 J.R.R. Tolkien
0909638 Fran Walsh
0101991 Philippa Boyens
0000704 Elijah Wood
0001557 Viggo Mortensen
0005212 Ian McKellen
0000233 Quentin Tarantino
0000812 Roger Avary
0000237 John Travolta
0000235 Uma Thurman
0000168 Samuel L. Jackson
0089217 Orlando Bloom
0001466 Sergio Leone
0898812 Luciano Vincenzoni
0408488 Agenore Incrocci
0

In [29]:
cursor.execute('select * from person;')
records = []
for item in cursor.fetchall() :
    print(item)
    records.append(item)

('0000005', 'Ingmar Bergman')
('0000006', 'Ingrid Bergman')
('0000007', 'Humphrey Bogart')
('0000008', 'Marlon Brando')
('0000012', 'Bette Davis')
('0000017', 'Marlene Dietrich')
('0000018', 'Kirk Douglas')
('0000020', 'Henry Fonda')
('0000021', 'Joan Fontaine')
('0000022', 'Clark Gable')
('0000023', 'Judy Garland')
('0000026', 'Cary Grant')
('0000027', 'Alec Guinness')
('0000032', 'Charlton Heston')
('0000033', 'Alfred Hitchcock')
('0000034', 'William Holden')
('0000036', 'Buster Keaton')
('0000037', 'Gene Kelly')
('0000038', 'Grace Kelly')
('0000040', 'Stanley Kubrick')
('0000041', 'Akira Kurosawa')
('0000044', 'Burt Lancaster')
('0000046', 'Vivien Leigh')
('0000048', 'Peter Lorre')
('0000051', 'James Mason')
('0000054', 'Marilyn Monroe')
('0000056', 'Paul Newman')
('0000059', 'Laurence Olivier')
('0000060', 'Gregory Peck')
('0000061', 'Tyrone Power')
('0000063', 'Anthony Quinn')
('0000064', 'Edward G. Robinson')
('0000071', 'James Stewart')
('0000075', 'Spencer Tracy')
('0000076', '

In [30]:
len(records)

1076

Insert Cast table data

In [32]:
with open('cast.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ',')
    for row in reader:
        print(row['movie_id'], row['person_id'])
        # insert

        sql_statement =('INSERT INTO `cast`'
                       '(`movie_id`,`person_id`)'
                        'VALUES(%(movie_id)s, %(person_id)s)')
        
        cursor.execute(sql_statement,row)
cnx.commit()

0111161 0000209
0111161 0000151
0111161 0348409
0068646 0000008
0068646 0000199
0068646 0001001
0468569 0000288
0468569 0005132
0468569 0001173
0071562 0000199
0071562 0000134
0071562 0000380
0050083 0000020
0050083 0002011
0050083 0000842
0108052 0000553
0108052 0000146
0108052 0001426
0167260 0000704
0167260 0001557
0167260 0005212
0110912 0000237
0110912 0000235
0110912 0000168
0120737 0000704
0120737 0005212
0120737 0089217
0060196 0000142
0060196 0908919
0060196 0001812
0109830 0000158
0109830 0000705
0109830 0000641
0137523 0000093
0137523 0001570
0137523 0001533
0167261 0000704
0167261 0005212
0167261 0001557
1375666 0000138
1375666 0330687
1375666 0680983
0080684 0000434
0080684 0000148
0080684 0000402
0133093 0000206
0133093 0000401
0133093 0005251
0099685 0000134
0099685 0000501
0099685 0000582
0073486 0000197
0073486 0001221
0073486 0077720
0114369 0000151
0114369 0000093
0114369 0000228
0038650 0000071
0038650 0001656
0038650 0000859
0047478 0001536
0047478 0793766
0047478 

In [33]:
cursor.execute('select * from cast;')
records = []
for item in cursor.fetchall() :
    print(item)
    records.append(item)

(1, '0111161', '0000209')
(2, '0111161', '0000151')
(3, '0111161', '0348409')
(4, '0068646', '0000008')
(5, '0068646', '0000199')
(6, '0068646', '0001001')
(7, '0468569', '0000288')
(8, '0468569', '0005132')
(9, '0468569', '0001173')
(10, '0071562', '0000199')
(11, '0071562', '0000134')
(12, '0071562', '0000380')
(13, '0050083', '0000020')
(14, '0050083', '0002011')
(15, '0050083', '0000842')
(16, '0108052', '0000553')
(17, '0108052', '0000146')
(18, '0108052', '0001426')
(19, '0167260', '0000704')
(20, '0167260', '0001557')
(21, '0167260', '0005212')
(22, '0110912', '0000237')
(23, '0110912', '0000235')
(24, '0110912', '0000168')
(25, '0120737', '0000704')
(26, '0120737', '0005212')
(27, '0120737', '0089217')
(28, '0060196', '0000142')
(29, '0060196', '0908919')
(30, '0060196', '0001812')
(31, '0109830', '0000158')
(32, '0109830', '0000705')
(33, '0109830', '0000641')
(34, '0137523', '0000093')
(35, '0137523', '0001570')
(36, '0137523', '0001533')
(37, '0167261', '0000704')
(38, '0167

In [34]:
len(records)

750

Insert Crew table Data

In [35]:
with open('crew.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ',')
    for row in reader:
        print(row['movie_id'], row['person_id'],row['role'])
        # insert

        sql_statement =('INSERT INTO `crew`'
                       '(`movie_id`,`person_id`,`role`)'
                        'VALUES(%(movie_id)s, %(person_id)s, %(role)s)')
        
        cursor.execute(sql_statement,row)
cnx.commit()

0111161 0001104 Director
0111161 0000175 Writer
0111161 0001104 Writer
0068646 0000338 Director
0068646 0701374 Writer
0068646 0000338 Writer
0468569 0634240 Director
0468569 0634300 Writer
0468569 0634240 Writer
0468569 0275286 Writer
0071562 0000338 Director
0071562 0000338 Writer
0071562 0701374 Writer
0050083 0001486 Director
0050083 0741627 Writer
0108052 0000229 Director
0108052 0447745 Writer
0108052 0001873 Writer
0167260 0001392 Director
0167260 0866058 Writer
0167260 0909638 Writer
0167260 0101991 Writer
0110912 0000233 Director
0110912 0000233 Writer
0110912 0000812 Writer
0120737 0001392 Director
0120737 0866058 Writer
0120737 0909638 Writer
0120737 0101991 Writer
0060196 0001466 Director
0060196 0898812 Writer
0060196 0001466 Writer
0060196 0408488 Writer
0109830 0000709 Director
0109830 0343165 Writer
0109830 0744839 Writer
0137523 0000399 Director
0137523 0657333 Writer
0137523 0880243 Writer
0167261 0001392 Director
0167261 0866058 Writer
0167261 0909638 Writer
0167261 

In [36]:
cursor.execute('select * from crew;')
records = []
for item in cursor.fetchall() :
    print(item)
    records.append(item)

(1, '0111161', '0001104', 'Director')
(2, '0111161', '0000175', 'Writer')
(3, '0111161', '0001104', 'Writer')
(4, '0068646', '0000338', 'Director')
(5, '0068646', '0701374', 'Writer')
(6, '0068646', '0000338', 'Writer')
(7, '0468569', '0634240', 'Director')
(8, '0468569', '0634300', 'Writer')
(9, '0468569', '0634240', 'Writer')
(10, '0468569', '0275286', 'Writer')
(11, '0071562', '0000338', 'Director')
(12, '0071562', '0000338', 'Writer')
(13, '0071562', '0701374', 'Writer')
(14, '0050083', '0001486', 'Director')
(15, '0050083', '0741627', 'Writer')
(16, '0108052', '0000229', 'Director')
(17, '0108052', '0447745', 'Writer')
(18, '0108052', '0001873', 'Writer')
(19, '0167260', '0001392', 'Director')
(20, '0167260', '0866058', 'Writer')
(21, '0167260', '0909638', 'Writer')
(22, '0167260', '0101991', 'Writer')
(23, '0110912', '0000233', 'Director')
(24, '0110912', '0000233', 'Writer')
(25, '0110912', '0000812', 'Writer')
(26, '0120737', '0001392', 'Director')
(27, '0120737', '0866058', 'W

In [37]:
len(records)

835

In [38]:
with open('genre.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ',')
    for row in reader:
        print(row['movie_id'], row['genre'])
        # insert

        sql_statement =('INSERT INTO `genre`'
                       '(`movie_id`,`genre`)'
                        'VALUES(%(movie_id)s, %(genre)s)')
        
        cursor.execute(sql_statement,row)
cnx.commit()

0111161 Drama
0068646 Crime
0068646 Drama
0468569 Action
0468569 Crime
0468569 Drama
0071562 Crime
0071562 Drama
0050083 Crime
0050083 Drama
0108052 Biography
0108052 Drama
0108052 History
0167260 Action
0167260 Adventure
0167260 Drama
0110912 Crime
0110912 Drama
0120737 Action
0120737 Adventure
0120737 Drama
0060196 Adventure
0060196 Western
0109830 Drama
0109830 Romance
0137523 Drama
0167261 Action
0167261 Adventure
0167261 Drama
1375666 Action
1375666 Adventure
1375666 Sci-Fi
0080684 Action
0080684 Adventure
0080684 Fantasy
0133093 Action
0133093 Sci-Fi
0099685 Biography
0099685 Crime
0099685 Drama
0073486 Drama
0114369 Crime
0114369 Drama
0114369 Mystery
0038650 Drama
0038650 Family
0038650 Fantasy
0047478 Action
0047478 Drama
0102926 Crime
0102926 Drama
0102926 Thriller
0120815 Drama
0120815 War
0317248 Crime
0317248 Drama
0816692 Adventure
0816692 Drama
0816692 Sci-Fi
0118799 Comedy
0118799 Drama
0118799 Romance
0120689 Crime
0120689 Drama
0120689 Fantasy
0076759 Action
0076759 A

In [39]:
cursor.execute('select * from genre;')
records = []
for item in cursor.fetchall() :
    print(item)
    records.append(item)

(1, '0111161', 'Drama')
(2, '0068646', 'Crime')
(3, '0068646', 'Drama')
(4, '0468569', 'Action')
(5, '0468569', 'Crime')
(6, '0468569', 'Drama')
(7, '0071562', 'Crime')
(8, '0071562', 'Drama')
(9, '0050083', 'Crime')
(10, '0050083', 'Drama')
(11, '0108052', 'Biography')
(12, '0108052', 'Drama')
(13, '0108052', 'History')
(14, '0167260', 'Action')
(15, '0167260', 'Adventure')
(16, '0167260', 'Drama')
(17, '0110912', 'Crime')
(18, '0110912', 'Drama')
(19, '0120737', 'Action')
(20, '0120737', 'Adventure')
(21, '0120737', 'Drama')
(22, '0060196', 'Adventure')
(23, '0060196', 'Western')
(24, '0109830', 'Drama')
(25, '0109830', 'Romance')
(26, '0137523', 'Drama')
(27, '0167261', 'Action')
(28, '0167261', 'Adventure')
(29, '0167261', 'Drama')
(30, '1375666', 'Action')
(31, '1375666', 'Adventure')
(32, '1375666', 'Sci-Fi')
(33, '0080684', 'Action')
(34, '0080684', 'Adventure')
(35, '0080684', 'Fantasy')
(36, '0133093', 'Action')
(37, '0133093', 'Sci-Fi')
(38, '0099685', 'Biography')
(39, '0099

In [40]:
len(records)

628

# Close the Cursor

In [41]:
cursor.close() 

True